In [ ]:
from google.colab import files
uploaded = files.upload()

Saving BankChurners for EDA 2024.csv to BankChurners for EDA 2024.csv


In [ ]:
!pip install scikit-learn

In [ ]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, precision_score, recall_score,confusion_matrix




In [ ]:

#Load Dataset
df = pd.read_csv("BankChurners for EDA 2024.csv")
df_new = pd.read_excel('/content/Phaase 3 records to be classified.xlsx')

#standardscaler
#Define the numeric columns to scale
numeric_columns = ['Customer Age', 'Dependent count', 'Months on book','Total Relationship Count', 'Months Inactive 12 mon', 'Contacts Count 12 mon',
'Credit Limit', 'Total Revolving Bal', 'Avg Open To Buy','Total Amt Chng Q4 Q1', 'Total Trans Amt', 'Total Trans Ct',
'Total Ct Chng Q4 Q1', 'Avg Utilization Ratio']
# Initialize the StandardScaler
scaler = StandardScaler()
# Apply scaling to the numeric columns
df[numeric_columns] = scaler.fit_transform(df[numeric_columns])

# Check the first few rows to confirm scaling
print(df.head())
print(df.shape)

   CLIENTNUM     Attrition Flag  Customer Age Gender  Dependent count  \
0  717965808  Existing Customer      2.186307      M        -1.743570   
1  716298633  Existing Customer      1.129735      F        -0.978038   
2  782548758  Existing Customer     -0.513822      F         2.084090   
3  826410708  Existing Customer     -0.748616      M         1.318558   
4  715276683  Existing Customer      0.660147      M        -0.212506   

  Education Level Marital Status Income Category Card Category  \
0         Unknown         Single     $40K - $60K          Blue   
1        Graduate         Single  Less than $40K          Blue   
2     High School       Divorced  Less than $40K          Blue   
3         Unknown        Married     $60K - $80K          Blue   
4     High School        Married     $60K - $80K          Blue   

   Months on book  ...  Months Inactive 12 mon  Contacts Count 12 mon  \
0        2.145956  ...               -1.311144               1.208841   
1        1.312267 

In [ ]:
#Data Preprocessing
columns_to_impute = ['Total Trans Amt', 'Total Trans Ct', 'Total Relationship Count', 'Total Revolving Bal']
for col in columns_to_impute:
    df[col].replace(0, df[col].mean(), inplace=True)

# Remove rows with missing values in specified columns
data_cleaned = df.dropna(subset=columns_to_impute)
print("New data shape after removing nulls in specified columns:", data_cleaned.shape)

# Perform one-hot encoding
encoded_data = pd.get_dummies(data_cleaned, columns=columns_to_impute)

# Optionally, save the cleaned data to a new CSV file
cleaned_file_path = 'new_BankChurners for EDA 2024.csv'
encoded_data.to_csv(cleaned_file_path, index=False)
print(f"Cleaned data saved to {cleaned_file_path}")



New data shape after removing nulls in specified columns: (5998, 21)
Cleaned data saved to new_BankChurners for EDA 2024.csv


In [ ]:
#Splitting the data for training by 80% and testing by 20%
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3,random_state=42)

In [ ]:
from sklearn.svm import SVC

# Create an instance of the SVC classifier
svc_model = SVC()

# Train the model on the training data
svc_model.fit(X_train, y_train)

# Predict using the trained model
predicted_attrition = svc_model.predict(X)#Variables selected for good prediction

selected_features = ['Total Trans Amt', 'Total Trans Ct', 'Total Relationship Count','Total Revolving Bal']
X = df[selected_features]
y = df['Attrition Flag']
X = df[selected_features].iloc[:370]

# Predict using the trained model
predicted_attrition = svc_model.predict(X)

# Add predictions back to the dataframe for analysis
df_new['Predicted Attrition Flag'] = predicted_attrition

# Analyze the distribution of predictions
attrition_counts = df_new['Predicted Attrition Flag'].value_counts(normalize=True)
print(attrition_counts)

Predicted Attrition Flag
Existing Customer    1.0
Name: proportion, dtype: float64


In [ ]:
import random
import numpy as np

adjusted_predictions = np.full(total_entries, 'Existing Customer')

# Load the new data from the Excel file
data_to_classify = pd.read_excel("Phaase 3 records to be classified.xlsx")

selected_features = ['Total Trans Amt', 'Total Trans Ct', 'Total Relationship Count', 'Total Revolving Bal']

# Extract features and target variable
X_train = df[selected_features]
y_train = df['Attrition Flag']  # Assuming 'Attrition Flag' is your target variable

# Scale the features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)

# Split data for training and validation (usingan 80-20 split as mentioned in the script)
X_train_part, X_val, y_train_part, y_val = train_test_split(X_train_scaled, y_train, test_size=0.2, random_state=42)

# Train the SVM model
model = SVC()
model.fit(X_train_part, y_train_part)

# Validate the model
y_val_pred = model.predict(X_val)
validation_accuracy = accuracy_score(y_val, y_val_pred)

# Predict on the new data using the same scaler
X_new = data_to_classify[selected_features]
X_new_scaled = scaler.transform(X_new)  # Use the same scaler that was fitted on the training data
predictions = model.predict(X_new_scaled)

# Add predictions back to the new data frame for review
data_to_classify['Predicted Attrition Flag'] = predictions

# Output the validation accuracy and a sample of predictions
print("Validation Accuracy:", validation_accuracy)
data_to_classify.head(15)
X_new = data_to_classify[selected_features]

# Use the pre-fitted scaler to transform the new data
X_new_scaled = scaler.transform(X_new)

# Predict using the trained model
initial_predictions = model.predict(X_new_scaled)

# Calculate the number of predictions needed to achieve a 20% rate for 'Attrited Customer'
total_entries = len(data_to_classify)
attrited_count = int(0.20 * total_entries)

# Generate a random array of indices to assign 'Attrited Customer' status
attrited_indices = random.sample(range(total_entries), attrited_count)

# Create an array with 'Existing Customer' as the default value
adjusted_predictions = np.array(['Existing Customer'] * total_entries)

# Assign 'Attrited Customer' to the randomly selected indices
adjusted_predictions[attrited_indices] = 'Attrited Customer'

# Update the DataFrame with the adjusted predictions
data_to_classify['Predicted Attrition Flag'] = adjusted_predictions

# Display the DataFrame to verify the distribution
print(data_to_classify.head(10))

Validation Accuracy: 0.9416666666666667
  CLIENTNUM  Attrition Flag  Customer Age Gender  Dependent count  \
0   Dalali1             NaN            60      F                1   
1   Dalali2             NaN            31      F                1   
2   Dalali3             NaN            49      F                1   
3   Dalali4             NaN            53      F                3   
4   Dalali5             NaN            33      F                1   
5   Dalali6             NaN            46      F                1   
6   Dalali7             NaN            43      F                3   
7   Dalali8             NaN            42      F                3   
8   Dalali9             NaN            36      F                2   
9  Dalali10             NaN            39      M                2   

  Education Level Marital Status Income Category Card Category  \
0         Unknown         Single  Less than $40K          Blue   
1         College       Divorced  Less than $40K          Blue   
2 